### Libraries

In [1]:
import numpy as np
import pulp as pl
import pandas as pd 

# Specify the Power Network

In [2]:
n = 9 # number of nodes 

nodes = range(n) # enumerates all nodes 

In [3]:
network_name = "ieee" + str(n)
filename = network_name + ".xlsx"

# Read NN Data

In [4]:
# get a given instance 

PD1 = 90
PD2 = 100
PD3 = 125

PD = [PD1,PD2,PD3]


# neural network solution

PG1 = 86.56 
PG2 = 134.38*2
PG3 = 94.06

PG = [PG1,PG2,PG3]

# Network Data

## Bus Data

In [5]:
# load bus data

bus = pd.read_excel(filename,sheet_name="Bus")

bus.head()

,bus,type,Pd,Qd,Gs,Bs,area,Vm,Va,baseKV,zone,Vmax,Vmin
0,0,3,0,0,0,0,1,1,0,345,1,1.1,0.9
1,1,2,0,0,0,0,1,1,0,345,1,1.1,0.9
2,2,2,0,0,0,0,1,1,0,345,1,1.1,0.9
3,3,1,0,0,0,0,1,1,0,345,1,1.1,0.9
4,4,1,90,30,0,0,1,1,0,345,1,1.1,0.9


In [6]:
# Demand Nodes

D_buses = bus[bus["Pd"] != 0]["bus"] 

In [7]:
# stores demand values given to neural network in appropriate node positions

P_D = np.zeros(n)

P_D[D_buses] = PD 

## Generator Data

In [8]:
# load generator data

gen = pd.read_excel(filename,sheet_name="Gen")

gen.head()

,bus,Pg,Qg,Qmax,Qmin,Vg,mBase,status,Pmax,Pmin,...,Pc2,Qc1min,Qc1max,Qc2min,Qc2max,ramp_agc,ramp_10,ramp_30,ramp_q,apf
0,0,72.3,27.03,300,-300,1.040,100,1,250,10,...,0,0,0,0,0,0,0,0,0,0
1,1,163.0,6.54,300,-300,1.025,100,1,300,10,...,0,0,0,0,0,0,0,0,0,0
2,2,85.0,-10.95,300,-300,1.025,100,1,270,10,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Generator Nodes

G_buses = gen["bus"]

In [10]:
#stores generator values from neural network in appropriate node positions

P_G = np.zeros(n)

P_G[G_buses] = PG 

In [11]:
# store the maximum and minimum power vectors 

P_G_max = np.zeros(n)
P_G_max[gen["bus"]] = gen["Pmax"]

P_G_min = np.zeros(n)
P_G_min[gen["bus"]] = gen["Pmin"]

## Branch Data

In [12]:
# load the branch data 

branch = pd.read_excel(filename,sheet_name="Branch")

branch.head(n)

,fbus,tbus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax
0,0,3,0.0000,0.0576,0.000,250,250,250,0,0,1,-360,360
1,3,4,0.0170,0.0920,0.158,250,250,250,0,0,1,-360,360
2,4,5,0.0390,0.1700,0.358,150,150,150,0,0,1,-360,360
3,2,5,0.0000,0.0586,0.000,300,300,300,0,0,1,-360,360
4,5,6,0.0119,0.1008,0.209,150,150,150,0,0,1,-360,360
5,6,7,0.0085,0.0720,0.149,250,250,250,0,0,1,-360,360
6,7,1,0.0000,0.0625,0.000,250,250,250,0,0,1,-360,360
7,7,8,0.0320,0.1610,0.306,250,250,250,0,0,1,-360,360
8,8,3,0.0100,0.0850,0.176,250,250,250,0,0,1,-360,360


In [13]:
# define line-related vectors 

P_flow = branch["rateA"]
reactance = branch["x"]

admittance = 1 / reactance

line_from = branch["fbus"].tolist()
line_to = branch["tbus"].tolist()

L = len(P_flow) # number of lines 

In [14]:
# construct the bus-admittance matrix

B = np.zeros([n,n])


for line in range(L):
    
    i = line_from[line] 
    j = line_to[line] 
    
    B[i][j] = - admittance[line]
    B[j][i] = - admittance[line]

    
for node in nodes:
    
    B[node][node] = - (sum (B[node][j] for j in nodes))

## p.u. Conversion

In [15]:
# convert all vectors to per unit at 100 MVA base

base = pd.read_excel(filename,sheet_name="Base")
Base = base["Base"][0]

P_G = P_G / Base
P_G_min = P_G_min / Base
P_G_max = P_G_max / Base
P_D = P_D / Base
P_flow = P_flow / Base

# Solve the Problem

In [16]:
# create the optimization problem with the decision variables 

m = pl.LpProblem('Projection_Scheme', pl.LpMinimize)

x = pl.LpVariable.dicts(name="PG_Projected", indices=nodes,cat='Continuous')
theta = pl.LpVariable.dicts(name="Phase_Angle", indices=nodes,cat='Continuous')

U = pl.LpVariable.dicts(name="U", indices=nodes, cat='Continuous')

In [17]:
# objective function

m += ( pl.lpSum([U[i] for i in nodes]), 'L1 Norm') ;

In [18]:
# absolute value constraints

for i in nodes:
    m += x[i] - P_G[i] <= U[i]
    m += x[i] - P_G[i] >= - U[i]

In [19]:
# generation limits constraints 

for i in nodes:
    m += x[i] <= P_G_max[i]
    m += x[i] >= P_G_min[i]

In [20]:
# line flow constraints

for line in range(L):
    
    bound = reactance[line] * P_flow[line]
    i = line_from[line] 
    j = line_to[line] 
    
    m += theta[i] - theta[j] <= bound
    m += theta[i] - theta[j] >= - bound

In [21]:
# power balance constraints

for i in nodes:
    m  +=  x[i] - P_D[i] == pl.lpSum(B[i][j]*theta[j] for j in nodes) 

In [22]:
# reference phase angle

m += theta[0] == 0

In [23]:
# solve

status = m.solve()
pl.LpStatus[status]

'Optimal'

# Store Results

In [24]:
# display the results

theta_vector = np.zeros(n)
generation_vector = np.zeros(n)

for variable in m.variables():
    
    name_components = variable.name.split("_")
    index = int(name_components[-1])
    value = variable.varValue
    
    if name_components[0] == "Phase":
        theta_vector[index] = value 
    
    elif name_components[0] == "PG":
        generation_vector[index] = value 

In [25]:
# compare generation vectors:

for node in nodes:
    
    original = P_G[node] * Base
    projected = generation_vector[node] * Base
    
    if projected!=0:
        print("Node "+str(node+1)+":",np.round(original,2)," -> ", np.round(projected,2))

Node 1: 86.56  ->  86.56
Node 2: 268.76  ->  218.44
Node 3: 94.06  ->  10.0


In [26]:
# compute line flow in every line

power_flow = np.zeros(L)

for line in range(L):
    
    i = line_from[line] 
    j = line_to[line] 
    
    power_flow[line] = (theta_vector[i] - theta_vector[j]) / reactance[line]
    
    print(np.round(power_flow[line],2),P_flow[line])

0.87 2.5
0.55 2.5
-0.35 1.5
0.1 3.0
-0.25 1.5
-1.25 2.5
-2.18 2.5
0.94 2.5
-0.31 2.5


In [27]:
for node in nodes:
    print(np.round(theta_vector[node]*180/np.pi,2))

0.0
12.06
-2.02
-2.86
-5.76
-2.36
-0.92
4.24
-4.39
